In [ ]:
import os
! rm -rf ondewo-.*
! git clone https://github.com/ondewo/ondewo-s2t-client-python.git
! cd ondewo-s2t-client-python && git checkout tags/1.5.0 -b 1.5.0 && pip install -r requirements.txt  && git status

In [ ]:
os.getcwd()
! cd ondewo-s2t-client-python && python -m pip install .

In [ ]:
import argparse
import wave

from ondewo.s2t import speech_to_text_pb2
from ondewo.s2t.client.client import Client
from ondewo.s2t.client.client_config import ClientConfig
from ondewo.s2t.client.services.speech_to_text import Speech2Text
from ondewo.s2t.speech_to_text_pb2 import ListS2tPipelinesRequest, Speech2TextConfig

In [ ]:
AUDIO_FILE: str = "examples/audiofiles/sample_1.wav"
# credentials = grpc.ssl_channel_credentials(root_certificates=cert)

MAX_MESSAGE_LENGTH: int = 60000000
GRPC_HOST: str = "<ADD GRPC SERVER HERE>"
GRPC_PORT: str = "<ADD GRPC PORT HERE>"
CHANNEL: str = f"{GRPC_HOST}:{GRPC_PORT}"
cert="<ADD CERTIFICATE HERE>"


options = [
    ('grpc.max_send_message_length', MAX_MESSAGE_LENGTH),
    ('grpc.max_receive_message_length', MAX_MESSAGE_LENGTH),
]

# channel = grpc.secure_channel(CHANNEL, credentials, options=options)

config: ClientConfig = ClientConfig(
  host=GRPC_HOST,
  port=GRPC_PORT, 
  grpc_cert=cert)
    
print(config)
    
client: Client = Client(config=config, use_secure_channel=True)

In [11]:
s2t_service: Speech2Text = client.services.speech_to_text
pipelines = s2t_service.list_s2t_pipelines(request=ListS2tPipelinesRequest()).pipeline_configs
print(pipelines)

In [ ]:
import wave 
with wave.open("ondewo-s2t-client-python/examples/audiofiles/sample_1.wav") as w:
    audio: bytes = w.readframes(w.getnframes())

In [ ]:
def find_pipeline_for_language(pipelines, language):
    for pipeline in pipelines:
        if pipeline.description.language == language:
            return pipeline

english_pipeline = find_pipeline_for_language(pipelines=pipelines, language='en')
german_pipeline = find_pipeline_for_language(pipelines=pipelines, language='de')

In [ ]:
pipeline: Speech2TextConfig = pipelines[1]

request = speech_to_text_pb2.TranscribeFileRequest(
    s2t_pipeline_id=pipeline.id,
    audio_file=audio,
    ctc_decoding=speech_to_text_pb2.CTCDecoding.BEAM_SEARCH_WITH_LM,
    )
# Send transcription request and get response
transcribe_response = s2t_service.transcribe_file(request=request)

print(f"File transcript: {transcribe_response.transcription}")